# Extraer datos de WEB

Este Script se concentra en extraer la informacion y datos de la galeria del museo Van Gogh y las cartas escritas por el mismo artista para guardar los archivos de texto, png y otros datos necesarios para crear el set de datos de entrenamiento para un modelo de Machine Larning. Se utiliza BeautifulSoup para extraer la informacion desde los portales WEB de interes

## Objetivos

Extraer las imagenes, datos y metadatos de cada elemento dentro de la coleccion del museoVan Gogh.

## Fuente de Datos

Las fuentes de datos para el conjunto de datos son:

* Galeria de obras y trabajo de Van Gogh: https://vangoghmuseum.nl/en/search/collection?q=&artist=Vincent%20van%20Gogh
* Archivo de cartas escritas por Van Gogh: http://vangoghletters.org/vg/search/simple?term=
* Ruta de carpetas de recursos de las cartas escritas por Van Gogh: http://vangoghletters.org/vg/letters/


In [11]:
# importar librerias necesarias para scrapyWEB
import re
import os
import urllib
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup

# importar librerias para hacer un dataframe de referencia
import numpy as np
import pandas as pd

# import os
# import urllib.request, urllib.error, urllib.parse
# from os import remove
# import os.path
# from os import remove
# from numpy import array
# from bs4 import BeautifulSoup
# import requests
# import requests, pandas, numpy, matplotlib.pyplot
# from bs4 import BeautifulSoup
# import re
# import urllib
# import wget

### Pagina Fuente (Root Page)

desde la pagina fuente recupero todos los enlaces a las obras de Vangohg y despues con la lista de enlaces recupero la informacion necesaria

Enlaces de interes:
- Extract links from webpage (BeautifulSoup): https://pythonspot.com/extract-links-from-webpage-beautifulsoup/
- How to: Find all tags with some given name and attributes: https://kite.com/python/examples/1734/beautifulsoup-find-all-tags-with-some-given-name-and-attributes
- Beautiful Soup can't find the part of the HTML I want: https://stackoverflow.com/questions/51982930/beautiful-soup-cant-find-the-part-of-the-html-i-want

EXTRA

## Galeria de Obras de vangoghmuseum.nl

### Definicion del DataFrame para la galeria
defino las columnas necesarias del DataFrame para saber que he podido recuperar y que no, TRUE es que si lo tengo, FALSE de lo contrario

In [12]:
# lista de columnas para trabajar el DataFrame de la galeria, sirve para manejar la calidad del proceso
# conocer los errores y ver que datos faltan por cada elemento de la galeria
soupCol = [
    "ID", # identificador unico de la galeria
    "ELEMENT_URL", # enlace del elemento recuperado del ScrapyWEB
    "NAME", # nombre del elemento en la galeria
    "DOWNLOAD_URL", # enlace 
    "HAS_DESCRIPTION", # booleano que identifica si se tiene la seccion de descripcion en el HTML del elemento
    "HAS_DOWNLOAD", # booleano que identifica si se tiene la seccion de enlace de descarga en el HTML del elemento
    "HAS_TAGS", # booleano que identifica si se tiene la seccion de tags de busqueda en el HTML del elemento
    "HAS_DATA", # booleano que identifica si se tiene la seccion de datos de archivo en el HTML del elemento
    "HAS_RELATEDW", # booleano que identifica si se tiene la seccion de trabajo relacionado en el HTML del elemento
    "ERR_", # si no se obtiene el nombre se guarda el error aca
    "ERR_DESCRIPTION", # si no se obtiene el nombre se guarda el error aca 
    "ERR_DOWNLOAD", # si no se obtiene el enlace de descarga se guarda el error aca 
    "ERR_TAGS", # si no se obtiene los tags de busqueda se guarda el error aca 
    "ERR_DATA", # si no se obtiene los datos de archivo se guarda el error aca 
    "ERR_RELATEDW", # si no se obtiene el trabajo relacionado se guarda el error aca 
]

In [13]:
# creo el dataFrame vacio con las columnas necesarias
soupFrame = pd.DataFrame(columns = soupCol)
soupFrame.head()

,ID,ELEMENT_URL,NAME,DOWNLOAD_URL,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,HAS_RELATEDW,ERR_,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW


### Creando el indice de la coleccion
creo el URL de busqueda en la pagina principal de la coleccion para extraer los URLs de los elementos creados por Van Gogh, para pruebas mantengo un numero limitado de URLs a recuperar.

In [14]:
# Se recorre la pagina principal y recuperan los enlaces de l 
# numero de paginas/obras/enlaces a iterar

paints = 9999
defaultPaints = 50

# maximo numero de pinturas conocidas
maxPaints = 1443

# busqueda base en la coleccion de Vangogh
paintsSearch = "https://vangoghmuseum.nl/en/search/collection?q=&artist=Vincent%20van%20Gogh&pagesize="
paintsRoot = "https://vangoghmuseum.nl"

# por defecto pruebo 50 elementos de la coleccion
paintsPage = paintsSearch + str(defaultPaints)

# si el numero de enlaces a buscar esta en el rango a propiado
if paints <= maxPaints and paints > 0:
    paintsPage = paintsSearch + str(paints)

# si el numero de enlaces es superior al maximo
elif paints > maxPaints:
    paintsPage = paintsSearch + str(maxPaints)

# si hay algo raro
else:
    paintsPage = paintsSearch + str(defaultPaints)

# reviso que cargue bien la pagina base
print("--- Search URL ---")
print(paintsPage)

--- Search URL ---
https://vangoghmuseum.nl/en/search/collection?q=&artist=Vincent%20van%20Gogh&pagesize=1443


In [15]:
# pido el HTML
# chequeo si el enlace sirve para sacar la informacion

# lista de enlaces, IDs nombres a elementos de la coleccion
links = list()
ids = list()
names = list()

# objetos necesarios para scrapy en beatifulsoup: handler y resultados
soup = None
linkSoup = list()
nameSoup = list()
# diccionario caracterisico para la busqueda para el enlace del elemento de la galeria
linkAttr = {"class":"link-teaser triggers-slideshow-effect", "href":re.compile("^/en/collection/")}
# diccionario caracterisico para la busqueda del nombre del elemento de la galeria
nameAttr = {"class":"text-base text-dark"}

try:
    # GET del URL
    pageReq = requests.get(paintsPage)

    # si el GET me responde bien con codigo 200
    if pageReq.status_code == 200:

        # uso beautifulSoup
        soup = BeautifulSoup(pageReq.content, "html.parser")

        # esta es la seccion del HTML donde estan los enlaces con tag 'a'
        linkSoup = soup.body.findAll("a", attrs = linkAttr)
        
        # esta es la seccion del HTML donde estan los enlaces con tag 'h3'
        nameSoup = soup.body.findAll("h3", attrs = nameAttr)

# si algo pasa aca esto esta muy mal
except Exception as e:
    print("Error in request: " + str(e))
    print("Status Code: " + str(pageReq.status_code))

In [16]:
# saco los links e IDs a las paginas que quiero dentro de la coleccion
# estan los elementos de tipo link-teaser y la expresion regular del href
for link in linkSoup:
    
    # reconstruyo el enlace completo
    # tempLink = paintsRoot + link.get('href')
    tempLink = urllib.parse.urljoin(paintsRoot, link.get("href"))
    tempID = link.get("href").replace("/en/collection/", "")
    ids.append(tempID)
    links.append(tempLink)

In [17]:
# saco los links e IDs a las paginas que quiero dentro de la coleccion
# estan los elementos de tipo link-teaser y la expresion regular del href
for name in nameSoup:
    
    # reconstruyo el enlace completo
    # tempLink = paintsRoot + link.get('href')
    tempName = re.sub("\s+", " ", name.string)
    tempName = tempName[1:-1]
    names.append(tempName)

In [18]:
# agregos los detalles del scrapy en las columnas del dataframe de calidad (nombres, IDs y URLs)
soupFrame["ID"] = ids
soupFrame["ELEMENT_URL"] = links 
soupFrame["NAME"] = names

In [19]:
#chequeo la informacion recuperada de la busqueda
print("Extracted links in search: " + str(len(links)))
print("Repeated link elements: " + str(len(links) != len(set(links))))

print("Extracted IDs in search: " + str(len(ids)))
print("Repeated IDs elements: " + str(len(ids) != len(set(ids))))

print("Extracted names in search: " + str(len(names)))
print("Repeated names elements: " + str(len(names) != len(set(names))))

print("Shape: " + str(soupFrame.shape))

soupFrame.info()
soupFrame.describe()

Extracted links in search: 1443
Repeated link elements: False
Extracted IDs in search: 1443
Repeated IDs elements: False
Extracted names in search: 1443
Repeated names elements: True
Shape: (1443, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1443 entries, 0 to 1442
Data columns (total 15 columns):
ID                 1443 non-null object
ELEMENT_URL        1443 non-null object
NAME               1443 non-null object
DOWNLOAD_URL       0 non-null object
HAS_DESCRIPTION    0 non-null object
HAS_DOWNLOAD       0 non-null object
HAS_TAGS           0 non-null object
HAS_DATA           0 non-null object
HAS_RELATEDW       0 non-null object
ERR_               0 non-null object
ERR_DESCRIPTION    0 non-null object
ERR_DOWNLOAD       0 non-null object
ERR_TAGS           0 non-null object
ERR_DATA           0 non-null object
ERR_RELATEDW       0 non-null object
dtypes: object(15)
memory usage: 169.2+ KB


,ID,ELEMENT_URL,NAME,DOWNLOAD_URL,HAS_DESCRIPTION,HAS_DOWNLOAD,HAS_TAGS,HAS_DATA,HAS_RELATEDW,ERR_,ERR_DESCRIPTION,ERR_DOWNLOAD,ERR_TAGS,ERR_DATA,ERR_RELATEDW
count,1443,1443,1443,0,0,0,0,0,0,0,0,0,0,0,0
unique,1443,1443,956,0,0,0,0,0,0,0,0,0,0,0,0
top,d0414-007V1962,https://vangoghmuseum.nl/en/collection/d0381V1962,"Head of a Woman, 1884 - 1885",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# guardando los links en TXT y CSV para backup del proceso y control de calidad
galleryIndex = "VanGoghGalleryIndex.txt"
galleryFile = "VanGoghGalleryIndex.csv"
dataFolder = "data"

# guardando archivo original de texto
# si no existe el directorio
if not os.path.exists(dataFolder):
    
    os.makedirs(dataFolder)

# si existe el directorio
else: 
    
    # sobreescribo el archivo siempre
    with open(os.path.join(os.getcwd(), dataFolder, galleryIndex), "w", encoding = "utf-8", errors = "ignore") as file:
        for link in links:
            file.write("%s\n" % link)
    
    soupFrame.to_csv(os.path.join(os.getcwd(), dataFolder, galleryFile), sep = ",", encoding = "utf-8", mode = "w", na_rep = "N.A.")
     
    

### Directorio y Carpetas para persistencia
configuro donde se va a guardar la informacion inicial dentro de OneDrive (gitHub no me deja), creo las carpetas y desacoplo el computador donde corre el script. Despues ejecuto los siguientes pasos:

- Creo el folder fuente "01 - Raw"
- Dentro del folder fuente creo los folderes "01 - Paints" y "02 - Letters"
- 

In [ ]:
# defino los directorios locales donde se va a persistir la informacion
# no olvidarse del cambiar "\" a "\\" en los filepath de windows por que si no no sirve nada
# dir de santiago
SFAM_ROOT = "C:\\Users\\Felipe\\OneDrive - Universidad de Los Andes\\03 - PhD\\04 - Clases\\05 - IA en Arte y Disenho\\03 - Proyecto\\01 - Data\\01 - Raw"
# dir de daniela
DCP_ROOT = ""
# variable intermedia para independizar, se comenta uno u otro segun donde se corra
WORK_ROOT = SFAM_ROOT
# WORK_ROOT = DCP_ROOT

# nombres de carpetas utiles donde se guarda la informacion
rawFolder = "01 - Raw"
paintsFolder = "01 - Paints"
lettersFolder = "02 - Letters"

#La ruta se obtiene con el dirpath y el filename y el dirpath.split(os.path.sep)[-1] agrega la clase
DATA_ROOT = os.path.dirname(WORK_ROOT)
print(DATA_ROOT)

In [ ]:
# preparar el directorio de trabajo
# si la ruta de trabajo local esta definida
workPath = ""

# si no eexiste el ROOT apropiado, esto lo debe arreglar el usuario
if not os.path.exists(WORK_ROOT):
    
    print("Make sure the WORK_ROOT is correct in your local HD!!!...")

# si si existe el root apropiado, configuro los folderes para guardar la informacion
elif os.path.exists(WORK_ROOT):

    # concateno los folders para la nueva carpeta de trabajo
    workPath = os.path.join(DATA_ROOT, rawFolder, paintsFolder)
#     workPath = os.path.join(DATA_ROOT, "rawFolder", "paintsFolder")

    # si todo va bien y necesito crear nuevo folder de trabajo
    if not os.path.exists(workPath):
        
        # creo folderes nuevos
        os.makedirs(workPath)
        print("creating proper folder configuration!!!...")
    
    # si los folderes ya existen
    if os.path.exists(workPath):
        
        print("Working folders and config are cool... carry on!...")


In [ ]:
# recorro el arreglo de enlaces habilitados para extraer la informacion
# variables de conteo del proceso
folderCount = 0
downloadCount = 0
linkCount = 0
repeatCount = 0

#inicio del ciclo para los enlaces
for link in links:
    
    # recojo la informacion necesaria para crear las carpetas y guarar los datos
    linkPar = urlparse(link)
    linkLen = len(linkPar.path.split("/"))
    linkFolder = linkPar.path.split("/")[linkLen-1]
        
    # chequeo si el enlace sirve para sacar la informacion
    try:
        # GET del URL
        linkReq = requests.get(link)
        
        # si el GET me responde bien con codigo 200
        if linkReq.status_code == 200:
            
            linkCount = linkCount + 1
                    
            # creo path para cada elemento en la coleccion
            tempPaintFolder = os.path.join(workPath, linkFolder)

            # creo la carpeta del nuevo elemento de la coleccion si no existe
            if not os.path.exists(tempPaintFolder):

                os.makedirs(tempPaintFolder)
                # cuento cuantos folderes nuevos creo
                folderCount = folderCount + 1

            # si existe la carpeta, descargo la imagen de la obra dentro de la coleccion de una vez
            if os.path.exists(tempPaintFolder):

                # parse del cuerpo del elemento de la coleccion
                linkSoup = BeautifulSoup(linkReq.content, "html.parser")

                # busco todos los elementos de tipo class="button dark-hover"
                soupDict = {"class":"button dark-hover", "href":re.compile("^/download/")}
                downloadSoup = linkSoup.find('a', attrs = soupDict)

                # creo el enlace para descargar la imagen
                tempLink = urllib.parse.urljoin(paintsRoot, downloadSoup.get("href"))

                # pido el enlace de la imagen
                downReq = requests.get(tempLink)

                # creo el nombre y la direccion del archivo que quiero guardar en la carpeta
                fileName = urlparse(tempLink)
                fileName = fileName.path.split("/")[len(fileName.path.split("/"))-1]
                filePath = os.path.join(tempPaintFolder, fileName)

                # si el archivo no existe lo guardo
                if not os.path.exists(filePath):
                    
#                     print("Downloding image from: " + str(tempLink) + " into: " + str(filePath))
                    print("Downloaded images: " + str(downloadCount))

                    with open(filePath, "wb") as file:
                        file.write(downReq.content)
                        file.close()
                        downloadCount = downloadCount + 1

                elif os.path.exists(filePath):
                    repeatCount = repeatCount + 1
#                     print("Repeated images: " + str(repeatCount))
            
    except Exception as e:
        print("Error in request: " + str(e))
        print("Status Code: " + str(linkReq.status_code))
        print("Status Code: " + str(downReq.status_code))
        print("Collection link: " + str(link))
        print("Image link: " + str(tempLink))
#         print("Explored links: " + str(linkCount))
        with open(filePath, "wb") as file:
            file.write(downReq.content)
            file.close()
            downloadCount = downloadCount + 1

print("Explored collection links: " + str(linkCount))
print("New folders created: " + str(folderCount))
print("Downloaded images: " + str(downloadCount))
print("Repeated images: " + str(repeatCount))

### Procesamiento de las pinturas en la galeria

el proceso sigue los siguientes pasos, para facilidad de la estructura se propone un formato JSON por cada uno de los frames que se desean guardar en formato TXT, al final cada carpeta debe tener 4 archivos TXT mas una image en formato PNG. Los pasos a seguir son los siguientes:

* Chequeo que los enlaces existan y creo las carpetas necesarias.
* Extraigo las anotaciones de busqueda cada uno de los elementos de la galeria desde el objeto "Search in the collection:".
* Extraigo los datos archivisticos de cada elemento de la galeria desde el objeto "OBJECT DATA".
* Extraigo los trabajos relacionados si existen de cada elemento de la galeria desde el objeto "Related work".
* Extraigo la imagen en formato PNG de cada elemento de la galeria en el desde el objeto "DOWNLOAD IMAGE"


 #### Enlaces utiles para el proceso
 
- Como chequear que un enlace esta vivo: https://stackoverflow.com/questions/51639585/checking-if-a-website-exist-with-python3
- Como utilizar libreria Request y el metodo GET: https://realpython.com/python-requests/
- Manejo de errores en Python 3 URL 1: https://www.python-course.eu/python3_exception_handling.php
- Manejo de errores en Python 3 URL 2: https://www.tutorialspoint.com/python3/python_exceptions.htm
- descargar una imagen desde un URL: https://stackabuse.com/download-files-with-python/

## Archivo de Cartas de vangoghletters.org

In [ ]:

totalpinturas=1443
temporal=199
pagina='https://vangoghmuseum.nl/en/collection/s'
a=np.arange(temporal)
lista=[]
for i in a:
    lista.append(pagina+str((i+1)).zfill(4)+'V1962')
    URL = lista[i]

    def iterate_over_children(soup, texts):
        for child in soup:
            if child.string == None:
                iterate_over_children(child, texts)
            else:
                if child.string != '\n':
                    texts.append(child.string)

    # Realizamos la petición a la web
    req = requests.get(URL)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    status_code = req.status_code
    if status_code == 200:

        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text, "html.parser")

        # Obtenemos todos los divs donde están las entradas
        #entradas = html.find_all('div', {'class': 'page-unit'})
        article = html.find('section')
        texts = []
        iterate_over_children(article, texts)
        texts_cleaned = [] 
        for text in texts:
            cleaned_text = text.replace("\n", "").strip()
            texts_cleaned.append(cleaned_text)

    #Daniela Oject Data
        # Obtenemos todos los divs donde están las entradas
        entradas = html.find_all('div', {'class': 'download-buttons-container'})


        # Recorremos todas las entradas para extraer el título, autor y fecha
        for i, entrada in enumerate(entradas):
            # Con el método "getText()" no nos devuelve el HTML
            hola= entrada.find('a').get('href') 
            texts_cleaned.append(hola)
            
        #links del parrafo (puede ir a las cartas)
        link_p=[tag['href'] for tag in html.select('p a[href]')]
        #links de search in the collection
        link_c=[tag['href'] for tag in html.select('aside a[href]')]
            
        
        #print(texts_cleaned)
        os.mkdir(texts_cleaned[0])
        #print (texts_cleaned)
        # Recorremos todas las entradas para extraer el título, autor y fecha
        save_path =texts_cleaned[0]

        f=open(texts_cleaned[0]+'.txt', 'w')

        completeName = os.path.join(save_path,texts_cleaned[0]+'.txt')    
        
        f.close()
        remove(texts_cleaned[0]+'.txt')

        file1 = open(completeName, "w")
        
        tamanio = len(texts_cleaned)
        for i in range(0,len(texts_cleaned)):
            file1.write(texts_cleaned[i]+'\n')
            
        for i in range(0,len(link_p)):
            file1.write(link_p[i]+'\n')
        for i in range(0,len(link_c)):
            file1.write(link_c[i]+'\n')
        file1.close()
      
        print('Beginning file download with wget module')

        url = 'https://vangoghmuseum.nl'+texts_cleaned[tamanio-1]
        wget.download(url, texts_cleaned[0]+'\ ' +texts_cleaned[0]+'.jpg')

        # guardar-paginaweb.py

        respuesta = urllib.request.urlopen(URL)
        contenidoWeb = respuesta.read()

        pag = open(texts_cleaned[0]+'.html', 'wb')
        completeName2 = os.path.join(save_path,texts_cleaned[0]+'.html')    
        pag.close()
        file2 = open(completeName2, "wb")
        file2.write(contenidoWeb)
        file2.close()
        
        remove(texts_cleaned[0]+'.html')
    else:
        print ("Status Code %d" % status_code)


In [ ]:
pag.close()

In [ ]:
page = html.find('p').getText()
print(page)

In [ ]:

print('Beginning file download with wget module')

url = 'https://vangoghmuseum.nl/download/45214344-2680-447a-92d3-e34667866f59.jpg'
wget.download(url, 'Congregation Leaving the Reformed Church in Nuenen\daniela.jpg')


In [ ]:
file1.close()

In [ ]:


URL = "https://vangoghmuseum.nl/en/collection/s0199V1962"

def iterate_over_children(soup, texts):
    for child in soup:
        if child.string == None:
            iterate_over_children(child, texts)
        else:
            if child.string != '\n':
                texts.append(child.string)

# Realizamos la petición a la web
req = requests.get(URL)

# Comprobamos que la petición nos devuelve un Status Code = 200
status_code = req.status_code
if status_code == 200:

    # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
    html = BeautifulSoup(req.text, "html.parser")

    # Obtenemos todos los divs donde están las entradas
    #entradas = html.find_all('div', {'class': 'page-unit'})
    article = html.find('section')
    texts = []
    iterate_over_children(article, texts)
    texts_cleaned = [] 
    for text in texts:
        cleaned_text = text.replace("\n", "").strip()
        texts_cleaned.append(cleaned_text)
  #  print(texts_cleaned)
       
#Daniela Oject Data
    # Obtenemos todos los divs donde están las entradas
    entradas = html.find_all('div', {'class': 'download-buttons-container'})
    

    # Recorremos todas las entradas para extraer el título, autor y fecha
    for i, entrada in enumerate(entradas):
        # Con el método "getText()" no nos devuelve el HTML
        hola= entrada.find('a').get('href') 
        texts_cleaned.append(hola)
    print (texts_cleaned)
    # Recorremos todas las entradas para extraer el título, autor y fecha

#url_imagen = "https://golang.org/doc/gopher/appenginegophercolor.jpg" # El link de la imagen
#nombre_local_imagen = "go.jpg" # El nombre con el que queremos guardarla
#imagen = requests.get(url_imagen).content
#with open(nombre_local_imagen, 'wb') as handler:
#	handler.write(imagen)

else:
    print ("Status Code %d" % status_code)
    



In [ ]:
os.mkdir('Comedores de Patatas')

In [ ]:

save_path ='Comedores de Patatas'

f = open('daniela6.txt', 'w')

completeName = os.path.join(save_path,'daniela6.txt')         

file1 = open(completeName, "w")

file1.write("daniela carcamo")
file1.close()

# remove(completeName)


In [ ]:
# abre-paginaweb.py
url = 'https://www.vangoghmuseum.nl/en/collection/s0005V1962?v=1'

respuesta = urllib.request.urlopen(url)
contenidoWeb = respuesta.read()

print(contenidoWeb[0:300])

In [ ]:
# guardar-paginaweb.py

url = 'https://www.vangoghmuseum.nl/en/collection/s0005V1962?v=1'

respuesta = urllib.request.urlopen(url)
contenidoWeb = respuesta.read()

f = open('patatas.html', 'wb')
f.write(contenidoWeb)
f.close

In [ ]:
print (respuesta)